In [5]:
import pandas as pd
import numpy as np
import datetime as dt

from support import *
from features.feature_ts import genX
from experiment.algorithms.cluster_prep import *

In [13]:
X = genX([1994,2014], drop_0 = True) # Load dataframe

dropping all zero rows


In [14]:
X

0         1         2         3         4  \
ProfileID date                                                           
1         1994-06-18  3.691667  0.800000  0.508333  2.183333  0.741667   
          1994-06-19  0.808333  0.500000  2.075000  0.691667  0.325000   
          1994-06-20  1.058333  1.625000  0.341667  2.100000  1.225000   
          1994-06-21  0.425000  2.350000  0.550000  0.291667  2.100000   
          1994-06-22  0.741667  2.325000  0.583333  1.191667  1.225000   
...                        ...       ...       ...       ...       ...   
12029071  2014-09-29  0.400000  0.400000  0.193333  0.246667  1.733333   
          2014-09-30  0.400000  0.400000  0.400000  0.400000  0.400000   
          2014-10-01  0.380000  0.306667  0.280000  0.293333  0.273333   
          2014-10-02  0.293333  0.286667  0.286667  0.293333  1.433333   
          2014-10-03  0.353333  0.313333  0.253333  0.273333  0.386667   

                             5         6         7         8         9  ...  \
ProfileID date                                                          ...   
1         1994-06-18  0.658333  1.991667  0.700000  2.383333  5.033333  ...   
          1994-06-19  2.075000  0.691667  0.316667  3.075000  2.033333  ...   
          1994-06-20  1.058333  4.808333  4.091667  2.083333  0.475000  ...   
          1994-06-21  0.591667  4.066667  2.791667  0.816667  2.100000  ...   
          1994-06-22  0.575000  4.066667  3.350000  0.866667  2.808333  ...   
...                        ...       ...       ...       ...       ...  ...   
12029071  2014-09-29  2.886667  0.440000  0.340000  0.380000  0.313333  ...   
          2014-09-30  0.640000  0.620000  1.406667  1.566667  0.280000  ...   
          2014-10-01  1.166667  0.366667  2.533333  3.986667  0.626667  ...   
          2014-10-02  0.753333  0.200000  2.286667  0.340000  0.280000  ...   
          2014-10-03  0.213333  0.526667  0.500000  1.686667  0.473333  ...   

                            14        15        16        17        18  \
ProfileID date                                                           
1         1994-06-18  1.450000  1.133333  1.666667  3.916667  9.241667   
          1994-06-19  2.125000  2.616667  2.358333  1.141667  7.283333   
          1994-06-20  1.433333  2.641667  2.116667  5.541667  5.275000   
          1994-06-21  3.150000  1.825000  4.016667  1.833333  3.466667   
          1994-06-22  1.500000  0.933333  2.791667  6.291667  6.491667   
...                        ...       ...       ...       ...       ...   
12029071  2014-09-29  0.853333  0.773333  0.320000  0.406667  1.566667   
          2014-09-30  0.293333  0.233333  0.226667  0.360000  1.146667   
          2014-10-01  0.233333  0.280000  0.213333  0.280000  0.966667   
          2014-10-02  0.333333  0.246667  0.213333  0.340000  0.846667   
          2014-10-03  0.206667  0.220000  0.400000  0.380000  0.726667   

                            19         20         21        22        23  
ProfileID date                                                            
1         1994-06-18  3.275000   6.325000   2.816667  0.925000  2.025000  
          1994-06-19  5.391667   6.441667   9.716667  7.500000  0.841667  
          1994-06-20  6.533333   4.708333  10.433333  9.508333  4.408333  
          1994-06-21  8.875000  10.850000  12.908333  9.241667  2.625000  
          1994-06-22  4.458333  13.125000  10.750000  6.200000  1.350000  
...                        ...        ...        ...       ...       ...  
12029071  2014-09-29  0.686667   2.273333   1.620000  0.506667  0.400000  
          2014-09-30  0.980000   1.480000   1.886667  0.380000  0.293333  
          2014-10-01  1.726667   1.306667   3.000000  0.906667  0.340000  
          2014-10-02  2.200000   1.526667   0.646667  0.633333  0.340000  
          2014-10-03  0.660000   0.633333   0.686667  0.640000  0.360000  

[3252833 rows x 24 columns]

In [34]:
# Only look at weekdays
df = X.copy()
df.reset_index(inplace = True)

# extract weekdays
df['day_names'] = df.date.dt.day_name()

weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday']

df['daytype'] = df.day_names.where(~df.day_names.isin(weekdays), 'weekday')

df.drop(['day_names'], axis = 1, inplace = True)

df_weekdays = df[df['daytype'] == 'weekday'] # Create dataframe with only weekdays

df_weekdays.set_index(['ProfileID'], inplace=True)
df_weekdays.drop(['daytype'],axis = 1, inplace = True)

C:\Users\Jason\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [58]:
mean = df_weekdays.groupby(["ProfileID"]).mean()

In [59]:
mean

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
ProfileID,,,,,,,,,,,,,,,,,,,,,
1,1.515633,1.526000,1.495467,1.578933,1.592267,1.707367,2.914867,2.388133,1.627433,1.499667,...,1.666400,1.724233,1.648833,2.041833,2.668467,2.753067,3.354400,3.227667,2.188000,1.596167
2,0.103331,0.077344,0.085376,0.079581,0.084633,0.163034,0.502867,0.267203,0.081880,0.077109,...,0.074160,0.097628,0.073419,0.187143,0.588789,0.514128,0.309219,0.202151,0.146758,0.106258
3,4.084262,4.376966,4.508858,4.917673,5.358824,4.785464,5.025920,7.802009,5.579086,4.360126,...,4.945172,5.616917,5.877477,6.677918,8.322331,8.013238,9.159963,8.159741,7.255072,5.505038
4,2.806694,2.500443,2.318668,2.306894,2.327160,2.545860,3.216391,4.677617,5.697475,6.088738,...,3.800445,3.616715,3.438847,4.456252,6.094647,6.023656,5.047271,4.243180,4.115435,3.461407
5,2.419639,2.083623,1.870402,2.004403,2.039214,1.936089,8.196546,9.981691,7.376087,6.117403,...,3.822313,5.839963,7.851651,8.322798,7.444273,8.782118,10.497867,9.895269,6.744583,3.708753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12029049,0.351351,0.301802,0.293243,0.312162,0.311937,0.297297,0.326126,0.411036,0.420991,0.256081,...,0.479525,0.626255,0.750450,1.323649,1.073243,0.639046,0.395045,0.354054,0.339640,0.328051
12029058,11.464062,10.981250,11.016927,12.858854,17.958854,21.957552,19.259635,19.540538,21.022917,19.766875,...,19.084115,19.723437,23.393750,31.678437,42.024870,39.462500,31.863281,23.400521,15.386198,12.590885
12029062,6.063281,5.633854,5.719531,6.038281,9.015625,10.632813,11.700521,7.237847,6.882031,6.685260,...,9.062760,9.578385,11.961198,15.236302,18.016927,18.000521,15.026042,11.366667,8.240625,6.713542


In [41]:
c

,date,0,1,2,3,4,5,6,7,8,...,14,15,16,17,18,19,20,21,22,23
ProfileID,,,,,,,,,,,,,,,,,,,,,
427,1995-08-02,0.069091,0.084833,0.070000,0.070000,0.083750,1.078833,2.105000,2.285750,0.633250,...,0.062000,0.430250,0.366667,0.695250,2.727750,3.590083,2.462417,0.643667,0.068000,0.068167
427,1995-08-03,0.068545,0.069000,0.068333,0.068000,0.228667,5.301083,3.499167,5.394500,1.785583,...,0.072500,0.842167,0.126833,1.970000,3.142500,1.875250,1.164417,0.487250,0.068333,0.068833
427,1995-08-04,0.069000,0.069000,0.084917,0.079083,0.074917,0.069917,2.220333,3.651833,2.161750,...,2.139833,0.073667,2.427000,2.723000,6.499083,2.354250,1.199167,0.917833,0.198667,0.126667
427,1995-08-07,0.069000,0.101000,0.069917,0.069917,0.090000,0.070417,9.623500,3.945417,2.126917,...,1.164833,1.299000,0.826167,2.662000,2.346500,2.604083,3.121417,0.609083,0.078750,0.069000
427,1995-08-08,0.069545,0.070000,0.070000,0.070417,0.071000,2.784583,5.358667,1.888750,0.080500,...,0.062750,1.491333,2.334083,1.932667,7.394083,2.414833,1.073500,1.860167,0.069000,0.069333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427,1996-08-05,0.078182,0.078667,0.248333,0.302000,0.265167,0.265250,0.232750,7.869667,0.413833,...,1.813583,0.254417,1.666667,7.261083,5.011500,1.503833,1.332500,1.292167,0.468583,0.529500
427,1996-08-06,0.102818,0.235583,0.261917,0.245167,0.069000,0.069000,0.850000,4.639333,3.093167,...,8.709000,0.819000,1.898750,7.286000,8.068667,2.870750,1.997583,1.084417,0.509333,0.242083
427,1996-08-07,0.079818,0.105833,0.131917,0.101000,0.122333,4.612083,8.144833,2.746083,2.391500,...,0.082583,0.830083,1.920833,0.125833,2.222250,2.885583,6.116333,2.775750,4.671000,1.036167


In [26]:
df_weekdays

0         1         2         3         4  \
ProfileID date                                                           
1         1994-06-20  1.058333  1.625000  0.341667  2.100000  1.225000   
          1994-06-21  0.425000  2.350000  0.550000  0.291667  2.100000   
          1994-06-22  0.741667  2.325000  0.583333  1.191667  1.225000   
          1994-06-23  1.166667  1.883333  0.450000  0.541667  2.333333   
          1994-06-24  2.408333  0.591667  0.316667  2.341667  0.550000   
...                        ...       ...       ...       ...       ...   
12029071  2014-09-29  0.400000  0.400000  0.193333  0.246667  1.733333   
          2014-09-30  0.400000  0.400000  0.400000  0.400000  0.400000   
          2014-10-01  0.380000  0.306667  0.280000  0.293333  0.273333   
          2014-10-02  0.293333  0.286667  0.286667  0.293333  1.433333   
          2014-10-03  0.353333  0.313333  0.253333  0.273333  0.386667   

                             5         6         7         8         9  ...  \
ProfileID date                                                          ...   
1         1994-06-20  1.058333  4.808333  4.091667  2.083333  0.475000  ...   
          1994-06-21  0.591667  4.066667  2.791667  0.816667  2.100000  ...   
          1994-06-22  0.575000  4.066667  3.350000  0.866667  2.808333  ...   
          1994-06-23  0.341667  4.216667  2.341667  2.583333  0.450000  ...   
          1994-06-24  1.183333  3.783333  2.958333  1.458333  1.191667  ...   
...                        ...       ...       ...       ...       ...  ...   
12029071  2014-09-29  2.886667  0.440000  0.340000  0.380000  0.313333  ...   
          2014-09-30  0.640000  0.620000  1.406667  1.566667  0.280000  ...   
          2014-10-01  1.166667  0.366667  2.533333  3.986667  0.626667  ...   
          2014-10-02  0.753333  0.200000  2.286667  0.340000  0.280000  ...   
          2014-10-03  0.213333  0.526667  0.500000  1.686667  0.473333  ...   

                            14        15        16        17        18  \
ProfileID date                                                           
1         1994-06-20  1.433333  2.641667  2.116667  5.541667  5.275000   
          1994-06-21  3.150000  1.825000  4.016667  1.833333  3.466667   
          1994-06-22  1.500000  0.933333  2.791667  6.291667  6.491667   
          1994-06-23  2.666667  4.758333  2.391667  5.233333  3.250000   
          1994-06-24  5.400000  1.041667  4.083333  3.191667  2.216667   
...                        ...       ...       ...       ...       ...   
12029071  2014-09-29  0.853333  0.773333  0.320000  0.406667  1.566667   
          2014-09-30  0.293333  0.233333  0.226667  0.360000  1.146667   
          2014-10-01  0.233333  0.280000  0.213333  0.280000  0.966667   
          2014-10-02  0.333333  0.246667  0.213333  0.340000  0.846667   
          2014-10-03  0.206667  0.220000  0.400000  0.380000  0.726667   

                            19         20         21        22        23  
ProfileID date                                                            
1         1994-06-20  6.533333   4.708333  10.433333  9.508333  4.408333  
          1994-06-21  8.875000  10.850000  12.908333  9.241667  2.625000  
          1994-06-22  4.458333  13.125000  10.750000  6.200000  1.350000  
          1994-06-23  5.958333  10.466667  10.066667  5.016667  0.733333  
          1994-06-24  4.258333   2.725000   0.975000  1.216667  1.608333  
...                        ...        ...        ...       ...       ...  
12029071  2014-09-29  0.686667   2.273333   1.620000  0.506667  0.400000  
          2014-09-30  0.980000   1.480000   1.886667  0.380000  0.293333  
          2014-10-01  1.726667   1.306667   3.000000  0.906667  0.340000  
          2014-10-02  2.200000   1.526667   0.646667  0.633333  0.340000  
          2014-10-03  0.660000   0.633333   0.686667  0.640000  0.360000  

[2314005 rows x 24 columns]